# **Latihan Analisis Data Menggunakan Google Colab**

Latihan ini merupakan bagian pembelajaran semata untuk melihat implementasi coding/programming menggunakan Python dan Google Colab. Data diunduh dari https://www.kaggle.com/datasets/dikisahkan/transjakarta-transportation-transaction/data?select=dfTransjakarta.csv

Versi terakhir 3 November 2025.

## (1) Tahapan Persiapan

### Memuat library yang diperlukan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import folium

### Mengunggah dataset yang akan dipakai

In [ ]:
# Option A: direct upload
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('dfTransjakarta.csv')

Bila sudah menggunakan opsi A, opsi ini bisa diabaikan/dilewati.

In [ ]:
# Option B: if stored in Drive
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dfTransjakarta.csv')

In [ ]:
# Option C: Jika file ada di URL publik

url = 'https://example.com/path/to/dfTransjakarta.csv'
df = pd.read_csv(url)

print(f"Data shape: {df.shape}")
print(df.head())

### Memeriksa data yang sudah terunggah

In [ ]:
df.head()
df.info()
df.describe(include='all')

### Drop missing value

In [ ]:
df_droped = df.dropna()
df_droped.info()

## (2) Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) adalah proses memeriksa dan memahami data secara mendalam dengan menggunakan statistik dan visualisasi. Tujuannya adalah untuk menemukan pola, anomali, hubungan, dan struktur dalam data tanpa membuat asumsi awal. EDA membantu dalam merumuskan hipotesis, memvalidasi asumsi, dan memandu langkah-langkah analisis selanjutnya.

Langkah-langkah umum dalam EDA:

1.    Memahami Data: Melihat struktur data, jumlah baris dan kolom, tipe data, dll.

2.    Pembersihan Data: Menangani missing values, outlier, duplikat, dll.

3.    Analisis Statistik Deskriptif: Menghitung mean, median, modus, standar deviasi, dll.

4.    Visualisasi Data: Membuat plot seperti histogram, box plot, scatter plot, heatmap, dll.

5.    Menemukan Hubungan: Melihat korelasi antara variabel, mengidentifikasi pola, dll.



EDA sangat penting karena memberikan wawasan awal yang berharga sebelum membangun model machine learning atau melakukan analisis statistik yang lebih kompleks.

### Siapkan dataset yang sudah bersih untuk analisis

In [ ]:
# Kita buat duplikat data frame
df1 = df_droped.copy()

# View columns
df1.columns

# Preview dataset
df1.head()

### Konversi datetime dalam format waktu

In [ ]:
df1['tapInTime'] = pd.to_datetime(df1['tapInTime'])
df1['tapOutTime'] = pd.to_datetime(df1['tapOutTime'])

# Extract time and day features
df1['tapInHour'] = df1['tapInTime'].dt.hour
df1['tapOutHour'] = df1['tapOutTime'].dt.hour
df1['tapDay'] = df1['tapInTime'].dt.dayofweek
df1.head()

### Ubah tapday menjadi hari

In [ ]:
df1['tapDay'] = df1['tapDay'].replace({
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday',
    3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'
})
df1.head()

### Melihat demografi pengguna

In [ ]:
customer = df1[['payCardName', 'payCardBirthDate', 'payCardBank', 'payCardSex']]
customer.info()

Setiap kali pengguna melakukan transaksi (tap) mewakili satu trip perjalanan, dengan demikian pengguna bisa muncul datanya berulang kali.

### Menghapus duplikasi data pengguna

In [ ]:
customer = customer.drop_duplicates(subset='payCardName', keep='first', ignore_index=True)
customer.info()

### Menghitung usia pengguna

In [ ]:
# Data diperoleh pada 2023
customer['age'] = 2023 - customer['payCardBirthDate']
customer.head()

### Melihat kartu yang digunakan

In [ ]:
cust_bank = customer['payCardBank'].value_counts().reset_index(name='count')
cust_bank

Menampilkan dalam bar chart.

In [ ]:
ax = cust_bank.plot(x='payCardBank', kind='bar', color='skyblue')

# Add text labels
for i, v in enumerate(cust_bank['count']):
    ax.text(i, v, str(v), ha='center', va='bottom')

plt.grid()
plt.xlabel('Bank')
plt.ylabel('Count')
plt.title('Bank Card Distribution')
plt.show()

Menampilkan dalam pie chart.

In [ ]:
plt.pie(cust_bank['count'], labels=cust_bank['payCardBank'], autopct='%1.1f%%')
plt.axis('equal')
plt.title('Bank Card Distribution')
plt.show()

Menambahkan kolom persentase.


In [ ]:
cust_bank['percentage'] = (cust_bank['count'] / cust_bank['count'].sum()) * 100
cust_bank

### Distribusi usia pelanggan

In [ ]:
cust_age = customer[['payCardName', 'age']].groupby('age').size().reset_index(name='count')
cust_age

Menampilkan dalam line chart.

In [ ]:
ax = cust_age.plot(x='age', kind='line', color='blue')
plt.grid()
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Customer Age Distribution')
plt.show()

### Distribusi jenis kelamin pelanggan

In [ ]:
cust_sex = (
    customer.groupby(['payCardSex', 'age'])['payCardName']
    .count()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

cust_sex.groupby('payCardSex').size()

Visualisasikan dalam chart.

In [ ]:
plt.figure(figsize=(8,6))
for sex in cust_sex['payCardSex'].unique():
    subset = cust_sex[cust_sex['payCardSex'] == sex]
    plt.plot(subset['age'], subset['count'], label=sex)

plt.legend()
plt.title('Customer Age Distribution by Gender')
plt.xlabel('Age')
plt.ylabel('Count')
plt.grid()
plt.show()

### Menunjukkan distribusi jumlah transaksi (tap in/out) per jam.

In [ ]:
# Transactions per hour (tap-in)
trans_by_hour = df1.groupby('tapInHour').size().reset_index(name='tap_count')

plt.figure(figsize=(10,5))
plt.bar(trans_by_hour['tapInHour'], trans_by_hour['tap_count'], color='steelblue')
plt.xticks(range(0,24))
plt.xlabel('Hour of Day')
plt.ylabel('Number of Transactions')
plt.title('Total Tap-In Transactions by Hour')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

### Menunjukkan intensitas perjalanan per hari

In [ ]:
# Transactions by day of week
trans_by_day = df1.groupby('tapDay').size().reindex([
    'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'
]).reset_index(name='tap_count')

plt.figure(figsize=(8,5))
plt.bar(trans_by_day['tapDay'], trans_by_day['tap_count'], color='coral')
plt.xlabel('Day of Week')
plt.ylabel('Number of Transactions')
plt.title('Tap-In Transactions by Day of Week')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

## (3) Recency, Frequency, Monetary (RFM)/Customer Life Value (CLV) Analysis

Recency, Frequency, Monetary (RFM) Analysis adalah metode yang digunakan untuk menganalisis perilaku pelanggan berdasarkan tiga faktor:

-    Recency (R): Seberapa baru seorang pelanggan melakukan pembelian? Biasanya diukur dengan jarak hari sejak pembelian terakhir.

-    Frequency (F): Seberapa sering seorang pelanggan melakukan pembelian dalam suatu periode? Biasanya dihitung jumlah transaksi.

-    Monetary (M): Berapa banyak uang yang dihabiskan oleh pelanggan? Biasanya dijumlahkan total pembelian.

Dengan RFM, kita dapat mengelompokkan pelanggan ke dalam segmen-segmen yang berbeda, seperti:

-    Pelanggan terbaik (nilai R, F, M tinggi)

-    Pelanggan yang hampir hilang (R rendah, F dan M tinggi)

-    Pelanggan baru (R tinggi, F dan M rendah)

-    Dan lain-lain.

Customer Lifetime Value (CLV) Analysis adalah metrik yang mengukur total nilai yang dihasilkan oleh seorang pelanggan selama hubungannya dengan bisnis. CLV membantu perusahaan dalam mengalokasikan sumber daya untuk mempertahankan pelanggan yang bernilai tinggi. Beberapa Pendekatan CLV:

-    Historical CLV: Total keuntungan yang dihasilkan pelanggan sejauh ini.

-    Predictive CLV: Memprediksi nilai masa depan pelanggan berdasarkan data historis.

In [ ]:
# --- RFM / CLV per customer ---
# Asumsi: df1 memiliki 'tapOutTime' datetime dan 'payAmount' numeric.
# Jika 'tapOutTime' belum datetime, pastikan sebelumnya telah dikonversi.

# Referensi tanggal: gunakan tanggal terakhir dalam dataset (maksimum tapOutTime)
reference_date = df1['tapOutTime'].max()
print("Reference date (dataset max tapOutTime):", reference_date)

# Hitung Recency, Frequency, Monetary (CLV proxy)
rfm = (df1
       .groupby('payCardName')
       .agg(
           LastTransaction = ('tapOutTime','max'),
           Frequency = ('transID','count'),    # jika ada transID unik, gunakan itu; kalau tidak, ganti 'payCardName'
           Monetary = ('payAmount','sum')
       )
       .reset_index()
)

# Recency dalam hari
rfm['Recency'] = (reference_date - rfm['LastTransaction']).dt.days

# Tambahkan info demografis (age, bank, sex) dari 'customer' jika ada
if 'age' in customer.columns:
    rfm = rfm.merge(customer[['payCardName','age','payCardBank','payCardSex']], on='payCardName', how='left')
else:
    rfm = rfm.merge(customer[['payCardName','payCardBank','payCardSex']], on='payCardName', how='left')

# Urutkan untuk tinjauan
rfm = rfm.sort_values(by='Monetary', ascending=False).reset_index(drop=True)
rfm.head(10)

### Membuat RFM scoring

In [ ]:
# --- Scoring RFM (contoh: 5-bucket) ---
# Pastikan tidak ada nilai identik untuk pemotongan; gunakan rank() agar stabil
rfm['R_score'] = pd.qcut(rfm['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1]).astype(int)   # recency: lebih kecil => lebih baik => label tinggi
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

rfm['RFM_Score'] = rfm[['R_score','F_score','M_score']].sum(axis=1)
rfm['RFM_Segment'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

# contoh: lihat top dan bottom segmen
display(rfm.sort_values('RFM_Score', ascending=False).head(10))
display(rfm.sort_values('RFM_Score', ascending=True).head(10))

### Memperbaiki durasi perjalanan menggunakan timedelta

In [ ]:
# --- Trip duration (hours) akurat menggunakan timedelta ---
# Pastikan tapInTime dan tapOutTime sudah datetime
df1['tapInTime'] = pd.to_datetime(df1['tapInTime'], errors='coerce')
df1['tapOutTime'] = pd.to_datetime(df1['tapOutTime'], errors='coerce')

# Hitung durasi (dalam jam, float)
df1['tripDuration_hours'] = (df1['tapOutTime'] - df1['tapInTime']).dt.total_seconds() / 3600.0

# Periksa nilai negatif (indikasi overnight atau inkonsistensi)
neg_count = (df1['tripDuration_hours'] < 0).sum()
print("Count negative durations:", neg_count)

# Jika ada negative durations, tampilkan contoh untuk inspeksi
if neg_count > 0:
    display(df1[df1['tripDuration_hours'] < 0].head(10))

# Statistik durasi
display(df1['tripDuration_hours'].describe().round(3))

### Rangkuman rute, route counts, top routes

In [ ]:
# --- Agregasi rute (tapInStopsName -> tapOutStopsName) ---
route_counts = (df1
                .groupby(['tapInStopsName','tapOutStopsName'])
                .size()
                .reset_index(name='TransactionCount')
                .sort_values('TransactionCount', ascending=False)
                .reset_index(drop=True))

route_counts.head(20)

### Hitung berapa banyak tap di tiap halte (tap in/out) dan tampilkan top 5

In [ ]:
# Tap in counts
tapIn_counts = df1.groupby('tapInStopsName').size().reset_index(name='tapInCounts').sort_values('tapInCounts', ascending=False).reset_index(drop=True)
tapIn_counts.head(10)

# Tap out counts
tapOut_counts = df1.groupby('tapOutStopsName').size().reset_index(name='tapOutCounts').sort_values('tapOutCounts', ascending=False).reset_index(drop=True)
tapOut_counts.head(10)

### Kapan waktu terpadat untuk top 5 halte (tap-in dan tap-out)

In [ ]:
# Pastikan kolom jam tersedia
df1['tapInHour'] = df1['tapInTime'].dt.hour
df1['tapOutHour'] = df1['tapOutTime'].dt.hour

# Top 5 tap-in stops
top5_in = tapIn_counts.head(5)['tapInStopsName'].tolist()

# Busiest hour per stop (tap-in) untuk top5
busiest_in = (df1[df1['tapInStopsName'].isin(top5_in)]
              .groupby(['tapInStopsName','tapInHour'])
              .size()
              .reset_index(name='count')
              .sort_values(['tapInStopsName','count'], ascending=[True, False])
              .drop_duplicates(subset='tapInStopsName', keep='first')
              .sort_values('count', ascending=False)
             )
busiest_in

# Untuk tap-out: top5 tap-out stops
top5_out = tapOut_counts.head(5)['tapOutStopsName'].tolist()

busiest_out = (df1[df1['tapOutStopsName'].isin(top5_out)]
               .groupby(['tapOutStopsName','tapOutHour'])
               .size()
               .reset_index(name='count')
               .sort_values(['tapOutStopsName','count'], ascending=[True, False])
               .drop_duplicates(subset='tapOutStopsName', keep='first')
               .sort_values('count', ascending=False)
              )
busiest_out

### Visualisasi sederhana (bar chart top 5 tap-in/tap-out)

In [ ]:
# Top5 tap-in bar
top5_in_df = tapIn_counts.head(5).copy()
ax = top5_in_df.plot(x='tapInStopsName', y='tapInCounts', kind='bar', legend=False)
for i, v in enumerate(top5_in_df['tapInCounts']):
    ax.text(i, v, str(v), ha='center', va='bottom')
plt.title('Top 5 Tap-In Stops')
plt.xlabel('')
plt.ylabel('Tap-In Count')
plt.grid(True)
plt.show()

# Top5 tap-out bar
top5_out_df = tapOut_counts.head(5).copy()
ax = top5_out_df.plot(x='tapOutStopsName', y='tapOutCounts', kind='bar', legend=False)
for i, v in enumerate(top5_out_df.iloc[:,1]):
    ax.text(i, v, str(v), ha='center', va='bottom')
plt.title('Top 5 Tap-Out Stops')
plt.xlabel('')
plt.ylabel('Tap-Out Count')
plt.grid(True)
plt.show()

### Gabungkan tap-in dan tap-out volume per halte

In [ ]:
station_volume = (
    pd.concat([
        df1[['tapInStopsName']].rename(columns={'tapInStopsName':'station'}),
        df1[['tapOutStopsName']].rename(columns={'tapOutStopsName':'station'})
    ])
    .value_counts()
    .reset_index(name='total_transactions')
    .rename(columns={'index':'station'})
)

station_volume_top10 = station_volume.head(10)

plt.figure(figsize=(10,5))
plt.barh(station_volume_top10['station'], station_volume_top10['total_transactions'], color='teal')
plt.xlabel('Total Transactions (Tap-In + Tap-Out)')
plt.ylabel('Station')
plt.title('Top 10 Busiest Stations')
plt.gca().invert_yaxis()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

### Peta Folium untuk Top Routes/Top Stops (top N)

In [ ]:
# --- Siapkan koordinat unik untuk setiap halte (gabungkan in/out coords jika tersedia) ---
coords_in = df1[['tapInStopsName','tapInStopsLat','tapInStopsLon']].drop_duplicates(subset='tapInStopsName').rename(columns={
    'tapInStopsName':'stopName','tapInStopsLat':'lat','tapInStopsLon':'lon'
})
coords_out = df1[['tapOutStopsName','tapOutStopsLat','tapOutStopsLon']].drop_duplicates(subset='tapOutStopsName').rename(columns={
    'tapOutStopsName':'stopName','tapOutStopsLat':'lat','tapOutStopsLon':'lon'
})

coords = pd.concat([coords_in, coords_out], ignore_index=True)
coords = coords.drop_duplicates(subset='stopName').reset_index(drop=True)

# Pilih top N routes (mis. top10 highest transaction count) dan ambil koordinat tapIn
top_routes = route_counts.head(10).copy()
top_routes = top_routes.merge(coords, left_on='tapInStopsName', right_on='stopName', how='left').rename(columns={'lat':'in_lat','lon':'in_lon'})
top_routes = top_routes.merge(coords, left_on='tapOutStopsName', right_on='stopName', how='left').rename(columns={'lat':'out_lat','lon':'out_lon'})

# Buat peta terpusat di Jakarta
m = folium.Map(location=[-6.1751,106.8272], zoom_start=11)

# Tambah marker dan polyline untuk tiap route (jika koordinat tersedia)
for _, r in top_routes.iterrows():
    if pd.notna(r['in_lat']) and pd.notna(r['in_lon']):
        folium.Marker([r['in_lat'], r['in_lon']],
                      popup=f"In: {r['tapInStopsName']}\nCount: {r['TransactionCount']}",
                      icon=folium.Icon(color='green')).add_to(m)
    if pd.notna(r['out_lat']) and pd.notna(r['out_lon']):
        folium.Marker([r['out_lat'], r['out_lon']],
                      popup=f"Out: {r['tapOutStopsName']}\nCount: {r['TransactionCount']}",
                      icon=folium.Icon(color='red')).add_to(m)
    # gambar garis rute jika kedua koordinat ada
    if pd.notna(r.get('in_lat')) and pd.notna(r.get('out_lat')):
        folium.PolyLine(locations=[[r['in_lat'], r['in_lon']],[r['out_lat'], r['out_lon']]],
                        weight=2, tooltip=f"{r['tapInStopsName']} → {r['tapOutStopsName']} ({r['TransactionCount']})"
                       ).add_to(m)

# Tampilkan peta (di notebook akan render)
m

### Deteksi outliers durasi (mis. > 6 jam) untuk memeriksa data rusak

In [ ]:
# gunakan df1_corrected jika koreksi tadi dijalankan; jika tidak, gunakan df1
df_check = df1_corrected if 'df1_corrected' in globals() else df1

# pastikan kolom ada
if 'tripDuration_hours' not in df_check.columns:
    df_check['tripDuration_hours'] = (df_check['tapOutTime'] - df_check['tapInTime']).dt.total_seconds() / 3600.0

# threshold outlier (adjustable)
threshold_hours = 6.0
outliers = df_check[df_check['tripDuration_hours'] > threshold_hours].copy()
print("Jumlah trip dengan durasi >", threshold_hours, "jam:", len(outliers))
print("Persentase dari total:", round(100*len(outliers)/len(df_check),4), "%")
display(outliers[['payCardName','tapInStopsName','tapOutStopsName','tapInTime','tapOutTime','tripDuration_hours']].head(20))

### Heatmap jam × halte (matrix) untuk melihat pola puncak harian

In [ ]:
df_heat = df_check.copy()
df_heat['tapInHour'] = df_heat['tapInTime'].dt.hour

# pilih top N stops berdasarkan total tap-in
top_n = 20
top_stops = df_heat['tapInStopsName'].value_counts().head(top_n).index.tolist()

# pivot table: index = stop, columns = hour, values = counts
pivot = (df_heat[df_heat['tapInStopsName'].isin(top_stops)]
         .groupby(['tapInStopsName','tapInHour'])
         .size()
         .reset_index(name='count')
        )

pivot_table = pivot.pivot(index='tapInStopsName', columns='tapInHour', values='count').fillna(0)

# sort stops by total volume for nicer plotting
pivot_table['total'] = pivot_table.sum(axis=1)
pivot_table = pivot_table.sort_values('total', ascending=False).drop(columns='total')

# plot heatmap
plt.figure(figsize=(14,10))
plt.imshow(pivot_table, aspect='auto', interpolation='nearest')
plt.colorbar(label='Tap-In Count')
plt.yticks(range(len(pivot_table.index)), pivot_table.index)
plt.xticks(range(0,24), range(0,24))
plt.xlabel('Hour of Day')
plt.title(f'Heatmap: Tap-In Counts per Hour for Top {top_n} Stops')
plt.show()

### Segmentasi RFM lebih lanjut (k-means clustering) pada fitur RFM (setelah scaling)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# gunakan rfm yang sudah ada; pastikan kolom Recency, Frequency, Monetary ada
rfm_model = rfm[['payCardName','Recency','Frequency','Monetary']].copy()

# log-transform Monetary & Frequency to reduce skew (add small const to avoid log(0))
rfm_model['log_M'] = np.log1p(rfm_model['Monetary'])
rfm_model['log_F'] = np.log1p(rfm_model['Frequency'])
rfm_model['R'] = rfm_model['Recency']  # keep as-is or transform if skewed

X = rfm_model[['R','log_F','log_M']].values

# scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Elbow method + silhouette for k in range
sse = []
sil_scores = []
K_range = range(2,9)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    sse.append(km.inertia_)
    sil_scores.append(silhouette_score(X_scaled, km.labels_))

# show results
print("K\tSSE\tSilhouette")
for k,s,e in zip(K_range,sse,sil_scores):
    print(k, round(s,2), round(e,3))

# choose k (example: pick k with good silhouette and elbow), here use k=4 as example
k_best = 4
km = KMeans(n_clusters=k_best, random_state=42, n_init=20)
labels = km.fit_predict(X_scaled)

rfm_model['cluster'] = labels
rfm_model['cluster'].value_counts()

# Centroids in original scaled space -> transform back for interpretation
centroids_scaled = km.cluster_centers_
centroids_unscaled = scaler.inverse_transform(centroids_scaled)
centroids = pd.DataFrame(centroids_unscaled, columns=['R','log_F','log_M'])
centroids['Frequency'] = np.expm1(centroids['log_F']).round(2)
centroids['Monetary'] = np.expm1(centroids['log_M']).round(2)
centroids[['R','Frequency','Monetary']]

# Cluster dengan low Recency, high Frequency, high Monetary = best customers.
# Cluster dengan high Recency, low Frequency, low Monetary = churned/low-value.
# Gunakan centroid values untuk memberi label (Champions, At-risk, Loyal, etc.).

In [ ]:
# install lifetimes if belum ada (jalankan sekali di Colab)
!pip install lifetimes

from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

# 1) persiapkan data transaksi: pastikan df1 punya kolom customer id dan transaction datetime & monetary
transactions = df1[['payCardName','tapOutTime','payAmount']].copy().rename(columns={
    'payCardName':'customer_id','tapOutTime':'date','payAmount':'monetary_value'
})
transactions = transactions.dropna(subset=['date'])  # ensure dates exist

# 2) summary table needed by lifetimes: uses weekly/days as frequency unit (use days)
summary = summary_data_from_transaction_data(transactions, 'customer_id', 'date', monetary_value_col='monetary_value', observation_period_end=transactions['date'].max())

# summary has: frequency, recency (in days), T (in days), monetary_avg
summary.head()

# 3) Fit BG/NBD model (for repeat transaction prediction)
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])
print(bgf.summary)

# 4) Fit Gamma-Gamma for monetary
ggf = GammaGammaFitter(penalizer_coef=0.0)
# lifetimes requires customers with frequency > 0 and monetary_avg > 0
summary_gg = summary[summary['monetary_value'] > 0].copy() # Use monetary_value
ggf.fit(summary_gg['frequency'], summary_gg['monetary_value']) # Use monetary_value
print(ggf.summary)

# 5) Predict CLTV (expected monetary value * expected repeat transactions) for horizon (e.g., 30 days)
expected_purchases_30 = bgf.conditional_expected_number_of_purchases_up_to_time(30, summary['frequency'], summary['recency'], summary['T'])
expected_avg_value = ggf.conditional_expected_average_profit(summary['frequency'], summary['monetary_value']) # Use monetary_value
clv_30 = expected_purchases_30 * expected_avg_value
summary['clv_30_days'] = clv_30
display(summary[['frequency','recency','T','monetary_value','clv_30_days']].sort_values('clv_30_days', ascending=False).head(10)) # Use monetary_value

## (4) Pipeline Modeling

Pipeline adalah cara untuk mengotomatiskan dan mengorganisir alur kerja (workflow) machine learning secara berurutan. Bayangkan seperti jalur perakitan di pabrik dimana data melewati berbagai tahap proses secara sistematis hingga akhirnya menghasilkan model.

Pipeline biasanya terdiri dari beberapa tahap:

1. Preprocessing & Feature Engineering

- Scaling/Normalization
- Handling missing values
- Encoding categorical variables
- Feature selection
- Dimensionality reduction (PCA)

2. Model Training

- Memilih algoritma (Random Forest, SVM, dll)
- Training model
- Hyperparameter tuning

3. Evaluation & Deployment
- Validasi model
- Testing
- Deployment ke production

In [ ]:
# --- 1. Safety checks & prepare CLV/RFM table ---
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# 1) Ensure datetime for safety (no import/cleaning repeated if already done)
df1['tapOutTime'] = pd.to_datetime(df1['tapOutTime'], errors='coerce')

# 2) reference date = last observed tapOutTime
reference_date = df1['tapOutTime'].max()
print("Reference date:", reference_date)

# 3) Frequency: prefer unique transID if exists, otherwise count rows
if 'transID' in df1.columns:
    freq = df1.groupby('payCardName')['transID'].nunique().reset_index(name='Frequency')
else:
    freq = df1.groupby('payCardName').size().reset_index(name='Frequency')

# 4) Monetary (Value): sum of payAmount (safe coercion to numeric)
df1['payAmount'] = pd.to_numeric(df1['payAmount'], errors='coerce').fillna(0)
monetary = df1.groupby('payCardName')['payAmount'].sum().reset_index(name='Value')

# 5) Last transaction date -> Recency (days since last transaction)
last_tx = df1.groupby('payCardName')['tapOutTime'].max().reset_index(name='LastTransaction')
last_tx['Recency'] = (reference_date - last_tx['LastTransaction']).dt.days

# 6) Merge into one CLV/RFM table
clv_simple = freq.merge(last_tx[['payCardName','Recency']], on='payCardName').merge(monetary, on='payCardName')

# 7) Final columns & quick check
clv_simple = clv_simple[['payCardName','Recency','Frequency','Value']].reset_index(drop=True)
clv_simple.head()

# 8) (Optional) scale for clustering (MinMaxScaler)
scaler = MinMaxScaler()
cols_to_scale = ['Recency','Frequency','Value']
clv_scaled = clv_simple.copy()
clv_scaled[cols_to_scale] = scaler.fit_transform(clv_scaled[cols_to_scale])

# show scaled sample
clv_scaled.head()

### Tentukan range cluster dan evaluasi dengan Elbow & Silhouette

Elbow Method adalah teknik yang digunakan untuk membantu menentukan jumlah kluster (K) yang optimal dalam algoritma K-Means. Metode ini didasarkan pada prinsip bahwa kita ingin memilih K yang meminimalkan variansi dalam kluster (dihitung dengan Within-Cluster Sum of Squares atau WCSS) tetapi tanpa menambahkan terlalu banyak kompleksitas (yaitu, tidak memilih K yang terlalu besar).

Disebut "elbow" karena plotnya sering menyerupai lengan, di mana terdapat penurunan WCSS yang cepat pada awalnya (seperti dari K=1 ke K=2, dan seterusnya) dan kemudian penurunan menjadi lebih landai setelah suatu titik. Titik di mana perubahan dari cepat ke landai itulah yang disebut "siku".

Silhouette Analysis adalah metode lain untuk mengevaluasi kualitas kluster dan menentukan K yang optimal. Metode ini mengukur seberapa baik setiap titik data dalam satu kluster cocok dengan klusternya sendiri dibandingkan dengan kluster terdekat lainnya.

Silhouette Analysis dapat memberikan ukuran yang lebih interpretatif tentang kualitas kluster. Ia juga dapat membantu membandingkan berbagai ukuran K dan juga berbagai algoritma kluster.

Kedua metode ini saling melengkapi. Dalam praktiknya, sering kali kita menggunakan kedua metode tersebut dan mempertimbangkan hasil keduanya untuk memilih K yang optimal. Selain itu, pertimbangan domain knowledge juga sangat penting.

In [ ]:
# --- Evaluasi jumlah cluster (Elbow & Silhouette) ---

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

X = clv_scaled[['Recency', 'Frequency', 'Value']]

ssd = []                # Sum of Squared Distances
silhouette_scores = []  # Silhouette Coefficient

range_n_clusters = range(2, 9)

for k in range_n_clusters:
    km = KMeans(n_clusters=k, random_state=42, n_init=20)
    km.fit(X)
    ssd.append(km.inertia_)
    silhouette_scores.append(silhouette_score(X, km.labels_))

# Plot hasil evaluasi
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(range_n_clusters, ssd, marker='o')
plt.xlabel('Jumlah Cluster (k)')
plt.ylabel('SSD (Inertia)')
plt.title('Elbow Curve')

plt.subplot(1,2,2)
plt.plot(range_n_clusters, silhouette_scores, marker='o', color='orange')
plt.xlabel('Jumlah Cluster (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis')
plt.tight_layout()
plt.show()

for k, s in zip(range_n_clusters, silhouette_scores):
    print(f"k={k} → Silhouette Score: {s:.3f}")

### Fit model K-Means dengan k terbaik

K-Means adalah sebuah algoritma unsupervised learning (pembelajaran tanpa pengawasan) yang digunakan untuk pengelompokan data (clustering). Tujuannya adalah untuk mengelompokkan data menjadi K buah kluster (kelompok), di mana setiap data dalam satu kluster memiliki karakteristik yang serupa, dan data dari kluster yang berbeda memiliki karakteristik yang berbeda.

Huruf "K" mewakili jumlah kluster yang ingin dibentuk, dan harus ditentukan oleh kita sebelum algoritma dijalankan.

In [ ]:
# --- Fit KMeans final ---
k_best = 4  # ubah sesuai hasil Elbow & Silhouette
km_final = KMeans(n_clusters=k_best, random_state=42, n_init=30)
clv_scaled['Cluster'] = km_final.fit_predict(X)

# Simpan label ke versi asli
clv_simple['Cluster'] = clv_scaled['Cluster']

# Lihat distribusi cluster
print("Distribusi jumlah pelanggan per cluster:\n")
print(clv_simple['Cluster'].value_counts().sort_index())

### Visualisasi hasil (2D PCA & 3D scatter)

In [ ]:
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

# --- PCA 2D projection untuk visualisasi ---
pca = PCA(n_components=2, random_state=42)
pca_coords = pca.fit_transform(X)

plt.figure(figsize=(8,6))
plt.scatter(pca_coords[:,0], pca_coords[:,1],
            c=clv_scaled['Cluster'], cmap='tab10', s=40, alpha=0.7)
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('Visualisasi Cluster (PCA 2D)')
plt.grid(True)
plt.show()

# --- 3D scatter (opsional) ---
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(clv_scaled['Recency'], clv_scaled['Frequency'], clv_scaled['Value'],
           c=clv_scaled['Cluster'], cmap='tab10', s=20)
ax.set_xlabel('Recency (scaled)')
ax.set_ylabel('Frequency (scaled)')
ax.set_zlabel('Value (scaled)')
ax.set_title('3D View: Cluster Pelanggan (R, F, V)')
plt.show()

### Profiling & interpretasi cluster

In [ ]:
# --- Profiling centroid & statistik per cluster ---
centroids_scaled = km_final.cluster_centers_
centroids_unscaled = pd.DataFrame(
    scaler.inverse_transform(centroids_scaled),
    columns=['Recency','Frequency','Value']
)
centroids_unscaled['Cluster'] = range(k_best)
centroids_unscaled = centroids_unscaled[['Cluster','Recency','Frequency','Value']]

print("Centroid tiap cluster (dalam skala asli):")
display(centroids_unscaled.round(2))

# Statistik ringkas per cluster
profile = (clv_simple.groupby('Cluster')
            .agg(Count=('payCardName','count'),
                 Recency_mean=('Recency','mean'),
                 Frequency_mean=('Frequency','mean'),
                 Value_mean=('Value','mean'))
            .sort_index())
display(profile.round(2))

### Boxplot distribusi per fitur

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
clv_simple.boxplot(column='Recency', by='Cluster')
plt.title('Recency per Cluster'); plt.suptitle('')

plt.subplot(1,3,2)
clv_simple.boxplot(column='Frequency', by='Cluster')
plt.title('Frequency per Cluster')

plt.subplot(1,3,3)
clv_simple.boxplot(column='Value', by='Cluster')
plt.title('Value per Cluster')

plt.tight_layout()
plt.show()

### Analisis naratif otomatis per cluster

In [ ]:
# Normalisasi agar dapat dibandingkan relatif antar-cluster
desc_stats = centroids_unscaled.copy()
desc_stats_norm = desc_stats[['Recency','Frequency','Value']].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
desc_stats_norm['Cluster'] = desc_stats['Cluster']

cluster_narratives = []

for _, row in desc_stats_norm.iterrows():
    cluster_id = int(row['Cluster'])
    rec, freq, val = row['Recency'], row['Frequency'], row['Value']

    # --- aturan deskriptif sederhana ---
    if rec < 0.3 and freq > 0.7 and val > 0.7:
        label = "Pelanggan sangat aktif dan bernilai tinggi (Champions)"
        mood = "🟢"
    elif rec < 0.5 and freq > 0.5:
        label = "Pelanggan aktif dengan nilai sedang (Regular Loyal)"
        mood = "🟡"
    elif rec > 0.7 and freq < 0.4:
        label = "Pelanggan dorman atau berisiko hilang (At Risk)"
        mood = "🔴"
    else:
        label = "Pelanggan sporadis / nilai rendah (Casual)"
        mood = "⚪"

    narrative = (f"Cluster {cluster_id}: {mood} "
                 f"Recency={rec:.2f}, Frequency={freq:.2f}, Value={val:.2f} → {label}.")
    cluster_narratives.append(narrative)

print("\n=== Narasi Otomatis per Cluster ===\n")
for n in cluster_narratives:
    print(n)

## (5) Forecasting Transaksi/Retensi (Probabilistik Model)

Pendekatan ini cocok untuk data seperti TransJakarta karena bersifat event-based (tiap tap = transaksi). Model paling umum: BG/NBD + Gamma-Gamma. Tujuannya memprediksi berapa kali pelanggan akan bertransaksi lagi (frequency prediction), dan memperkirakan nilai moneter masa depan (Customer Lifetime Value).

In [ ]:
# install lifetimes if belum ada (jalankan sekali)
!pip install lifetimes --quiet

from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes.plotting import plot_frequency_recency_matrix, plot_period_transactions
import pandas as pd

# --- Defensive: pastikan df1 ada dan kolom yang diperlukan tersedia ---
# Required columns: 'payCardName' (customer id), 'tapOutTime' (datetime), 'payAmount' (monetary)
# If tapOutTime missing for some rows, consider using tapInTime instead (but be consistent).

# 1) Prepare transactions dataframe
transactions = df1[['payCardName','tapOutTime','payAmount']].copy().rename(columns={
    'payCardName':'customer_id','tapOutTime':'date','payAmount':'monetary_value'
})
transactions = transactions.dropna(subset=['customer_id','date'])  # require id + date
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')
transactions = transactions.dropna(subset=['date'])  # drop rows with invalid dates

# Ensure monetary numeric and non-negative
transactions['monetary_value'] = pd.to_numeric(transactions['monetary_value'], errors='coerce').fillna(0)
transactions.loc[transactions['monetary_value'] < 0, 'monetary_value'] = 0

# 2) Build summary table required by lifetimes
# observation_period_end optional: use max date in transactions
observation_end = transactions['date'].max()
print("Observation period end:", observation_end)

summary = summary_data_from_transaction_data(
    transactions,
    customer_id_col='customer_id',
    datetime_col='date',
    monetary_value_col='monetary_value',
    observation_period_end=observation_end
)
# summary columns: frequency, recency, T, monetary_avg
# summary = summary.reset_index().rename(columns={'index':'CustomerID'}).set_index('CustomerID') # This renaming caused previous error, remove

# Quick checks
print("Summary shape:", summary.shape)
display(summary.head())

# 3) Filter customers usable for modeling
# BG/NBD requires frequency >= 0 (frequency = number of repeat purchases)
# Gamma-Gamma requires monetary_avg > 0
summary_for_bgnbd = summary.copy()   # BG/NBD can work with freq >=0 (but freq=0 means no repeat purchases)
summary_for_gg = summary[summary['monetary_value'] > 0].copy() # Use monetary_value here

# 4) Fit BG/NBD (Beta-Geometric / Negative Binomial)
bgf = BetaGeoFitter(penalizer_coef=0.001)
# Note: arguments are (frequency, recency, T)
bgf.fit(summary_for_bgnbd['frequency'], summary_for_bgnbd['recency'], summary_for_bgnbd['T'])
print("BG/NBD fitted. params:", bgf.params_)

# Optional: visualize frequency-recency matrix (diagnostic)
# plot_frequency_recency_matrix(bgf) # Move plotting to a separate cell
# plot_period_transactions(bgf) # Move plotting to a separate cell

# 5) Fit Gamma-Gamma model (monetary), requires customers with monetary_avg > 0 and frequency > 0
ggf = GammaGammaFitter(penalizer_coef=0.001)
# lifetimes requires customers with frequency > 0 and monetary_avg > 0
# Use monetary_value for the monetary data in Gamma-Gamma fitting
ggf.fit(summary_for_gg['frequency'], summary_for_gg['monetary_value'])
print("Gamma-Gamma fitted. params:", ggf.params_)

# 6) Make predictions
# Predict expected purchases in next 30 days
summary['predicted_purchases_30d'] = bgf.conditional_expected_number_of_purchases_up_to_time(
    30,
    summary['frequency'],
    summary['recency'],
    summary['T']
)

# Predict expected average transaction value (for customers with monetary_avg > 0)
# For those with monetary_value == 0, we can leave NaN or impute.
summary.loc[summary['monetary_value'] > 0, 'expected_avg_value'] = ggf.conditional_expected_average_profit(
    summary.loc[summary['monetary_value'] > 0, 'frequency'],
    summary.loc[summary['monetary_value'] > 0, 'monetary_value']
)

# 7) Predict CLV: use lifetimes' customer_lifetime_value (returns expected total monetary value)
# Here we compute 3-month CLV (time=3 months), freq='D' because our recency/T are in days
# Need to ensure frequency and monetary_value are passed correctly
summary['predicted_clv_3m'] = ggf.customer_lifetime_value(
    bgf,
    summary['frequency'],
    summary['recency'],
    summary['T'], # Pass T as well
    summary['monetary_value'].fillna(0), # Pass monetary value, handle NaNs
    time=3*30,       # time in days (3 months * 30 days/month)
    freq='D',     # frequency of the data: 'D' for days
    discount_rate=0.01
)


# 8) Inspect top customers by predicted CLV
top_clv = summary.sort_values('predicted_clv_3m', ascending=False).head(10)
display(top_clv[['frequency','recency','T','monetary_value','predicted_purchases_30d','predicted_clv_3m']])

# 9) Merge predictions back to original CLV table (optional)
# Assuming clv_simple has 'payCardName' as a regular column
clv_simple_reset = clv_simple.reset_index() # Reset index to merge on 'payCardName'
summary_reset = summary.reset_index().rename(columns={'customer_id': 'payCardName'}) # Reset index and rename for merge

clv_with_preds = clv_simple_reset.merge(
    summary_reset[['payCardName', 'predicted_purchases_30d', 'predicted_clv_3m']],
    on='payCardName',
    how='left'
)
display(clv_with_preds[['payCardName','Recency','Frequency','Value','predicted_purchases_30d','predicted_clv_3m']].head(10))

In [ ]:
import matplotlib.pyplot as plt
from lifetimes.plotting import plot_frequency_recency_matrix, plot_period_transactions

# Pastikan summary ada dan memiliki kolom 'T'
print("Max T (observation period in days):", summary['T'].max())

# 1) Frequency-Recency matrix — T harus skalar (mis. maksimum T dari summary)
T_scalar = summary['T'].max()

plt.figure(figsize=(8,6))
plot_frequency_recency_matrix(bgf, T_scalar)   # bgf + scalar T
plt.title('Frequency–Recency Matrix (BG/NBD)')
plt.show()

# 2) Period transactions plot — visualisasi model fit vs actual
# plot_period_transactions can be called with the model alone
plt.figure(figsize=(8,6))
plot_period_transactions(bgf)
plt.title('Model Fit vs Actual Transactions (BG/NBD)')
plt.show()

### Forecasting Volume Lalu Lintas (Time Series) dengan ARIMA/SARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import pandas as pd

# Agregasi transaksi mingguan
ts = df1.set_index('tapInTime').resample('W')['transID'].count()
ts = ts.fillna(0)

# Split data train-test (e.g., last 4 weeks for testing)
test_size = 4
train = ts[:-test_size]
test = ts[-test_size:]

print(f"Length of train data: {len(train)}")
print(f"Length of test data: {len(test)}")

# Fit SARIMA model
# Choose appropriate order and seasonal order based on EDA (example values)
# (p,d,q) order and (P,D,Q,S) seasonal order
# Seasonal period S=52 for weekly data if a year of data is present
# For this dataset, S=4 might be more appropriate given the data span (monthly)
try:
    model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
    result = model.fit(disp=False)

    # Forecast test_size steps ahead
    forecast = result.get_forecast(steps=test_size)
    forecast_ci = forecast.conf_int()

    plt.figure(figsize=(10, 5))
    plt.plot(train, label='Train')
    plt.plot(test, label='Test')
    plt.plot(forecast.predicted_mean, label='Forecast', color='red')
    plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.3)
    plt.legend()
    plt.title('Forecasting Volume Penumpang Mingguan')
    plt.show()

except Exception as e:
    print(f"Error during SARIMA fitting or forecasting: {e}")
    print("Consider adjusting the SARIMA order or seasonal order if the error persists.")

### Prediksi Churn/Aktivitas Pelanggan (Machine Learning)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Ensure clv_simple is available and has the necessary columns
if 'clv_simple' not in globals():
    print("Error: 'clv_simple' DataFrame not found. Please run the previous cells to generate it.")
else:
    # Merge age from customer DataFrame
    # Assuming 'customer' DataFrame with 'payCardName' and 'age' exists from previous steps
    if 'customer' in globals() and 'age' in customer.columns:
        clv_demo = clv_simple.merge(customer[['payCardName','age']], on='payCardName', how='left')
    else:
         print("Warning: 'customer' DataFrame or 'age' column not found. Proceeding without age.")
         clv_demo = clv_simple.copy()


    # Define features (X) and target (y)
    # Check if 'age' was successfully merged before including it
    features = ['Recency','Frequency','Value']
    if 'age' in clv_demo.columns:
        features.append('age')

    X = clv_demo[features]

    # Example definition of churn proxy: Customers who have not made a transaction recently (e.g., Recency > median Recency)
    y = (clv_demo['Recency'] > clv_demo['Recency'].median()).astype(int)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Fit RandomForestClassifier model
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Evaluate model
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
print(scores, scores.mean())

import pandas as pd
pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)

Secara statistik, model Anda bekerja sempurna pada dataset uji saat ini.

Namun, hasil ini terlalu bagus untuk jadi kenyataan — kemungkinan besar ada kebocoran data atau overfitting.

Perlu validasi silang waktu atau cross-validation untuk memastikan performa tersebut stabil.

## Lain-lain

Analisis lain-lain masih dalam tahap beta (uji coba). Comments and suggestions are welcome.

### Origin–Destination (OD) Network Analysis

Identify the most frequent passenger movement patterns between stops (e.g., home–work corridors) to reveal passenger flow hotspots for route optimization or congestion management.

In [ ]:
import networkx as nx

edges = (df1.groupby(['tapInStopsName', 'tapOutStopsName'])
           .size().reset_index(name='weight'))

G = nx.from_pandas_edgelist(edges, 'tapInStopsName', 'tapOutStopsName', edge_attr='weight', create_using=nx.DiGraph())

# Centrality metrics
centrality = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G, weight='weight')

# Top “hub” stops
sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10]

### Customer Mobility Typology (Behavioral Clustering)

In [ ]:
user_trips = (df1.groupby('payCardName')
                 .agg(trips=('transID','count'),
                      unique_stops=('tapInStopsName','nunique'),
                      mean_trip_hours=('tripDuration_hours','mean'),
                      weekdays=('tapDay','nunique'))
                 .reset_index())
display(user_trips.head()) # Add display to show the result

### Bank/Payment Analytics

In [ ]:
bank_stats = (df1.groupby('payCardBank')
                .agg(avg_spent=('payAmount','mean'),
                     trips=('transID','count'),
                     unique_users=('payCardName','nunique'))
                .reset_index())
bank_stats.plot(kind='bar', x='payCardBank', y='avg_spent')

### Spatial Density Mapping

In [ ]:
import folium
from folium.plugins import HeatMap

heat_data = df1[['tapInStopsLat','tapInStopsLon']].values.tolist()
m = folium.Map(location=[-6.1751,106.8272], zoom_start=11)
HeatMap(heat_data, radius=10).add_to(m)
m

### Origin–Destination Network + HeatMap overlay

In [ ]:
import pandas as pd
import networkx as nx
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

# Defensive copy
od_df = df1.copy()

# Ensure stop name columns exist
required_cols = ['tapInStopsName','tapOutStopsName',
                 'tapInStopsLat','tapInStopsLon','tapOutStopsLat','tapOutStopsLon']
missing_cols = [c for c in required_cols if c not in od_df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in df1: {missing_cols}")

# 1) Aggregate OD flows (weight = number of trips)
od_agg = (od_df
          .groupby(['tapInStopsName','tapOutStopsName',
                    'tapInStopsLat','tapInStopsLon','tapOutStopsLat','tapOutStopsLon'])
          .size()
          .reset_index(name='weight')
         )

# Drop self-loops if you want (optional)
od_agg = od_agg[od_agg['tapInStopsName'] != od_agg['tapOutStopsName']].reset_index(drop=True)

# 2) Build directed NetworkX graph
G = nx.DiGraph()
for _, row in od_agg.iterrows():
    src = row['tapInStopsName']
    dst = row['tapOutStopsName']
    w = int(row['weight'])
    # add nodes with coords as attributes (keep first encountered coords)
    if src not in G.nodes:
        G.add_node(src, lat=row['tapInStopsLat'], lon=row['tapInStopsLon'])
    if dst not in G.nodes:
        G.add_node(dst, lat=row['tapOutStopsLat'], lon=row['tapOutStopsLon'])
    # if edge already exists, sum weights (shouldn't if grouped)
    if G.has_edge(src, dst):
        G[src][dst]['weight'] += w
    else:
        G.add_edge(src, dst, weight=w)

print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")

# 3) Compute simple centrality measures (degree and betweenness)
degree_cen = dict(G.degree(weight='weight'))       # weighted degree (in+out)
in_deg = dict(G.in_degree(weight='weight'))
out_deg = dict(G.out_degree(weight='weight'))
betw = nx.betweenness_centrality(G, weight='weight', normalized=True)

# Create dataframe for nodes ranking
nodes_df = pd.DataFrame([
    {'stop': n,
     'lat': G.nodes[n].get('lat'),
     'lon': G.nodes[n].get('lon'),
     'degree': degree_cen.get(n,0),
     'in_degree': in_deg.get(n,0),
     'out_degree': out_deg.get(n,0),
     'betweenness': betw.get(n,0)}
    for n in G.nodes()
])
nodes_df = nodes_df.sort_values('degree', ascending=False).reset_index(drop=True)
nodes_df.head(10)

# 4) Extract top K edges by weight for mapping
top_k = 30   # adjustable: how many top OD links to draw
edges_df = pd.DataFrame([ (u, v, d['weight']) for u,v,d in G.edges(data=True) ],
                        columns=['source','target','weight'])
edges_top = edges_df.sort_values('weight', ascending=False).head(top_k).reset_index(drop=True)
edges_top['source_lat'] = edges_top['source'].map(nodes_df.set_index('stop')['lat'])
edges_top['source_lon'] = edges_top['source'].map(nodes_df.set_index('stop')['lon'])
edges_top['target_lat'] = edges_top['target'].map(nodes_df.set_index('stop')['lat'])
edges_top['target_lon'] = edges_top['target'].map(nodes_df.set_index('stop')['lon'])
edges_top = edges_top.dropna(subset=['source_lat','source_lon','target_lat','target_lon']).reset_index(drop=True)

# 5) Build Folium map centered on Jakarta (or centroid of stops)
if nodes_df[['lat','lon']].dropna().shape[0] > 0:
    center = [nodes_df['lat'].median(), nodes_df['lon'].median()]
else:
    center = [-6.1751,106.8272]  # fallback
m = folium.Map(location=center, zoom_start=11, tiles='Cartodb Positron')

# 6) Add HeatMap for tap-in locations (all tap-in events, weighted by one)
heat_data = od_df[['tapInStopsLat','tapInStopsLon']].dropna().values.tolist()
HeatMap(heat_data, radius=8, blur=12, max_zoom=13).add_to(m)

# 7) Add top OD polylines with thickness proportional to weight
max_w = edges_top['weight'].max() if not edges_top.empty else 1
min_w = edges_top['weight'].min() if not edges_top.empty else 1
for _, r in edges_top.iterrows():
    line = [(r['source_lat'], r['source_lon']), (r['target_lat'], r['target_lon'])]
    # line thickness scaled (tweak multiplier as needed)
    weight = 1 + 8 * ((r['weight'] - min_w) / (max_w - min_w + 1e-9))
    folium.PolyLine(locations=line,
                    weight=weight,
                    color='crimson',
                    opacity=0.7,
                    tooltip=f"{r['source']} → {r['target']}: {r['weight']} trips"
                   ).add_to(m)
    # add small circle markers for endpoints for clarity
    folium.CircleMarker(location=(r['source_lat'], r['source_lon']),
                        radius=2,
                        color='blue',
                        fill=True,
                        fill_opacity=0.6).add_to(m)
    folium.CircleMarker(location=(r['target_lat'], r['target_lon']),
                        radius=2,
                        color='green',
                        fill=True,
                        fill_opacity=0.6).add_to(m)

# 8) Add a layer with top nodes labeled (optional)
for _, nr in nodes_df.head(20).iterrows():
    folium.CircleMarker([nr['lat'], nr['lon']],
                        radius=4 + (nr['degree'] / (nodes_df['degree'].max()+1))*6,
                        color='darkblue',
                        fill=True, fill_opacity=0.8,
                        popup=f"{nr['stop']} (deg={int(nr['degree'])}, bw={nr['betweenness']:.3f})").add_to(m)

# 9) Save or display map
# In Colab this will render; use m.save('od_network_map.html') to export
m